<a href="https://colab.research.google.com/github/tanvigorre/GardenAPIs/blob/main/AppleClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start importing the libraries

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 ! ls "/content/drive/MyDrive/Data"

ls: cannot access '/content/drive/MyDrive/Data': No such file or directory


Read the file flower.csv into a dataframe that we call df

In [ ]:
cc.run_app(app=app)

INFO:     Started server process [60]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://837c-35-229-222-21.ngrok.io" -> "http://localhost:12000"


In [ ]:
data_1 = '/content/drive/My Drive/Garden App/App Models/Apple'
train = '/content/drive/My Drive/Garden App/App Models/Apple/train'
validation = '/content/drive/My Drive/Garden App/App Models/Apple/valid'

In [3]:
target_size = 200
data_labels = ['Apple Black rot', 'Apple Healthy', 'Apple Scab','Cedar apple rust']
labels = []
images = []
class_locations = {'Apple Black rot':0,
                   'Apple Healthy':1,
                   'Apple Scab':2,
                   'Cedar apple rust':3}

In [5]:
import os
from tqdm import tqdm 
from PIL import Image
from numpy import asarray
target_data = 400
for x in data_labels:
  path = os.path.join('/content/drive/My Drive/Garden App/App Models/Apple/train', x)
  for j in tqdm(os.listdir(path)):
    img_path = os.path.join(path,j)
    img = Image.open(img_path).convert("RGB")
    img = img.resize((target_size, target_size))
    img = asarray(img)
    images.append(img)
    labels.append(x)

100%|██████████| 10/10 [00:04<00:00,  2.08it/s]


In [6]:
import os
from tqdm import tqdm 
from PIL import Image
from numpy import asarray
for x in data_labels:
  path = os.path.join('/content/drive/My Drive/Garden App/App Models/Apple/valid', x)
  for j in tqdm(os.listdir(path)):
    img_path = os.path.join(path,j)
    img = Image.open(img_path).convert("RGB")
    img = img.resize((target_size, target_size))
    img = asarray(img)
    images.append(img)
    labels.append(x)

100%|██████████| 14/14 [00:03<00:00,  3.89it/s]


In [7]:
import numpy as np
x = np.array(images)
y = np.array(labels)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [9]:
from tensorflow.keras.utils import to_categorical
y_cat_train = []
for i in y_train:
  y_cat_train.append(data_labels.index(i))
y_cat_train = to_categorical(y_cat_train)

y_cat_test = []
for i in y_test:
  y_cat_test.append(data_labels.index(i))
y_cat_test = to_categorical(y_cat_test)

In [10]:
x_test = x_test/255
x_train = x_train/255

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [11]:
#Create structure of model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu', input_shape = (200, 200, 3), padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (2,2), activation = 'relu', input_shape = (200, 200, 3), padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (1,1), activation = 'relu', padding = 'valid'))
model.add(Conv2D(filters = 4, kernel_size = (3,3), activation = 'relu', padding = 'valid'))

model.add(Activation('softmax'))
model.add(GlobalAveragePooling2D())
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [12]:
early_stop = EarlyStopping(monitor='val_loss',patience=1)

In [13]:
model.fit(x_train,y_cat_train,epochs=5,validation_data=(x_test,y_cat_test))

Epoch 1/5
44/44 [==============================] - 248s 6s/step - loss: 0.5824 - accuracy: 0.9697 - val_loss: 0.2345 - val_accuracy: 0.9683
Epoch 2/5
44/44 [==============================] - 246s 6s/step - loss: 0.1972 - accuracy: 0.9697 - val_loss: 0.2012 - val_accuracy: 0.9683
Epoch 3/5
44/44 [==============================] - 246s 6s/step - loss: 0.1777 - accuracy: 0.9697 - val_loss: 0.1801 - val_accuracy: 0.9683
Epoch 4/5
44/44 [==============================] - 252s 6s/step - loss: 0.1606 - accuracy: 0.9697 - val_loss: 0.1538 - val_accuracy: 0.9683
Epoch 5/5
44/44 [==============================] - 253s 6s/step - loss: 0.1409 - accuracy: 0.9697 - val_loss: 0.1319 - val_accuracy: 0.9683


In [14]:
from keras.models import load_model

model.save('apple_model.h5')
del model

model = load_model('apple_model.h5')

In [15]:
model.save('applemodel.h5')

In [18]:
import pickle

with open('apple_pkl.pkl', 'wb') as files:
  pickle.dump(model, files)

INFO:tensorflow:Assets written to: ram://07a4d8a7-afdb-45e8-b49a-aa939e49d64c/assets


INFO:tensorflow:Assets written to: ram://07a4d8a7-afdb-45e8-b49a-aa939e49d64c/assets


In [19]:
pickle.load(open("apple_pkl.pkl", "rb"))

In [ ]:
import pickle
import logging
from fastapi import FastAPI
from PIL import Image

model = None

@app.on_event("startup")
def load_model():
  global model
  model = pickle.load(open("model_apple.pkl", "rb"))

@app.post("/api", tags=["prediction"])
def home():
  img = #figure this out
  img = img.resize((200,200))
  img = asarray("img")
  pred = model.predict(img)


In [ ]:
model.fit(train)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['accuracy','val_accuracy']].plot()